### Minhash con PySpark

In [ ]:
import pyspark
# Carga ufnciones extra
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('minhash').getOrCreate()

# Minhash y texto
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col, size
import string
import nltk
nltk.download('stopwords')

# Graficar
import seaborn as sns

Podemos definir vectores dispersos en PySpark usando estos parametros de la siguiente manera

In [ ]:
vector = Vectors.sparse(10, [0, 5, 8], [1, 3, 2])
vector.toArray()



Podemos crear un DataFrame con vectores dispersos de la siguiente manera


In [ ]:
matriz_dispersa = [(Vectors.sparse(6, [0, 1, 2], [1.0, 1.0, 1.0]),),
                   (Vectors.sparse(6, [2, 3, 4], [1.0, 1.0, 1.0]),),
                   (Vectors.sparse(6, [0, 2, 4], [1.0, 1.0, 1.0]),)]

matriz_dispersa = spark.createDataFrame(matriz_dispersa)

matriz_dispersa.show()

In [ ]:
documentos = spark.createDataFrame([(0, 'a,b,b,c'.split(',')),
                                    (1, 'b,c,d'.split(',')),
                                    (2, 'a,c'.split(','))],
                                   ['id', 'palabras'])

documentos.show()

In [ ]:
cv = CountVectorizer(inputCol  = 'palabras', 
                     outputCol = 'features', 
                     vocabSize = 6, 
                     minDF     = 1)

modelo = cv.fit(documentos)
mat_rep = modelo.transform(documentos)

mat_rep.show(truncate = False)

#### Datos

Los datos que usaremos son de la serie Rick y Morty. Corresponde a transcripciones de los dialogos de los personajes. La base de datos cuenta con los siguientes campos:

    Indice (Identificadore unico del renglon en la base de datos)
    Temporada
    Numero de episodio
    Nombre del episodio
    Linea de dialogo

Carguemos los datos en un DataFrame de PySpark

In [ ]:
df_rym = spark.read.csv('content/RickAndMortyScripts.csv', header = True, inferSchema = True)

In [ ]:
df_rym.printSchema()

In [ ]:
df_rym.show(5)

Para fines del analisis que llevaremos a cabo en este notebook solo haremos uso del as columnas del nombre y el dialogo del personaje

In [ ]:
df_rym = df_rym.select('index', 'name', 'line')
df_rym.show(5)

Para hacer aun mas simple el analisis quedemonos solo con los dialogos de los personajes principales

In [ ]:
personajes_principales = ['Rick', 'Morty', 'Beth', 'Jerry', 'Summer']
df_rym = df_rym.where(df_rym.name.isin(personajes_principales))

Revisemos la distribucion de dialogos de los personajes que tienen mas dialogos

In [ ]:
df_dist_dialog = df_rym.groupBy('name') \
                       .count() \
                       .toPandas()

sns.barplot( x = 'name', 
             y = 'count', 
             data = df_dist_dialog)

#### Preprocesamiento de los datos

##### Limpieza del texto



Aqui realizaremos el proceso que ya hemos hecho anteriormente, el cual consiste en:

    Transformar todo el texto a minusculas
    Eliminar signos de puntuacion
    Eliminar las palabras vacias, que son palabras que aportan poco contexto al problema

Veamos como se encuentra el texto originalmente

In [ ]:
df_rym.limit(3) \
      .toPandas() \
      .loc[0:3, 'line']

In [ ]:
# Hacemos minusculas
df_rym = df_rym.rdd \
               .map(lambda x: (x[0], x[1], x[2].lower())) \
               .toDF(['id', 'nombre', 'dialog']) 

# Vemos como va el proceso
df_rym.limit(3) \
      .toPandas() \
      .loc[0:3,'dialog']

In [ ]:
# Elimina signos de puntacion
df_rym = df_rym.rdd \
           .map(lambda x: (x[0], x[1], x[2].translate(str.maketrans('', '', string.punctuation)))) \
           .toDF(['id', 'nombre', 'dialog']) 

# Vemos como va el proceso
df_rym.limit(3) \
      .toPandas() \
      .loc[0:3,'dialog']

Quitar stop words

In [ ]:
# Definimos funcion para quitar stop words
palabras_vacias = nltk.corpus.stopwords.words('english')

def quita_palabras_vacias(texto):
  texto_limpio = [ palabra for palabra in texto if palabra not in palabras_vacias ]
  return texto_limpio

# Quitamos stop words
df_rym = df_rym.rdd \
           .map(lambda x: (x[0], x[1], x[2].split(' '))) \
           .map(lambda x: (x[0], x[1], quita_palabras_vacias(x[2]))) \
           .toDF(['id', 'nombre', 'dialog']) \
           .filter(size('dialog') > 0) # Elimina texto vacio

# Vemos como va el proceso
df_rym.limit(7) \
      .toPandas() \
      .loc[0:7,'dialog']

#### Matriz de representacion

Ahora necesitamos representar los datos como una matriz de representacion, la cual nos dice que elementos del vocabulario aparecen en cada elemento del conjunto de datos.

In [ ]:
cv = CountVectorizer(inputCol  = 'dialog', 
                     outputCol = 'features',
                     binary    = True, # Solo llena con 0 o 1
                     vocabSize = 3000,  # Tamano maximo del vocabulario 
                     minDF     = 1)    # En cuantos docs diferentes debe
                                       # aparecer una palabra para ser 
                                       # considerada para el vocabulario

In [ ]:
# Calculamos el vocabulario dados los datos
matriz_representacion = cv.fit(df_rym)
# Traducimos los datos la matriz de representacion
df_rym_mr = matriz_representacion.transform(df_rym)
# Mustramos
df_rym_mr.show(10)

#### Minhash y distancia de Jaccard

Para crear un modelo basado en Minhashing hacemos lo siguiente

In [ ]:
minhash = MinHashLSH(inputCol      = 'features', 
                     outputCol     = 'hashes', 
                     numHashTables = 100)

modelo_mh = minhash.fit(df_rym_mr)

Podemos mostrar como se ve la signature matrix creada a partir de la matriz de representacion

In [ ]:
df_rym_mh = modelo_mh.transform(df_rym_mr) 

df_rym_mh.limit(10) \
         .show()

#### Similitud entre lineas

Podemos calcular las distancias entre los elementos de dos dataframes

In [ ]:
similitud_entre_lineas = modelo_mh.approxSimilarityJoin(datasetA  = df_rym_mh,  
                                                      datasetB  = df_rym_mh, 
                                                      threshold = 0.5, 
                                                      distCol   = 'dist_jaccard') \
                                    .select(col('datasetA.nombre').alias('nombre_1'), 
                                        col('datasetB.nombre').alias('nombre_2'), 
                                        col('dist_jaccard')) \
                                    .where(col('dist_jaccard') > 0) # Quitamos las que son iguales

similitud_entre_lineas.show(4)      

Veamos cuantas si pudimos identificar los dialogos por personaje

In [ ]:
coincidencias = similitud_entre_lineas.withColumn('coinciden', 
                                                    col('nombre_1') == col('nombre_2')) \
                                        .groupBy('coinciden') \
                                        .count() \
                                        .toPandas()

sns.barplot( x    = 'coinciden', 
             y    = 'count', 
             data = coincidencias)

#### Vecinos mas cercanos

Calculamos los elementos mas cercanos a un elemento dado.

Extraemos una linea correspondinete a Morty

In [ ]:
[renglon] = df_rym_mr.where(df_rym_mr.id == 170).collect()
print(renglon)
linea = renglon[3]
print(linea)

Buscamos los elementos mas cercanos

In [ ]:
vecinos_cercanos = modelo_mh.approxNearestNeighbors(dataset             = df_rym_mh, 
                                                    key                 = linea, 
                                                    numNearestNeighbors = 50, 
                                                    distCol             = 'dist_jaccard') \
                            .where(col('dist_jaccard') > 0) # Quitamos la linea repetida

Veamos cuantas lineas corresponden a cada personaje

In [ ]:
vecinos_cercanos_hist = vecinos_cercanos.groupby('nombre') \
                                        .count() \
                                        .toPandas()

In [ ]:
sns.barplot( x    = 'nombre', 
             y    = 'count', 
             data = vecinos_cercanos_hist)

### Ejercicio

Haga alguna variacion del problema aqui presentado y explique sus resultados. Algunas de las cosas que podria hacer son:

    Cambiar el conjutno de personajes considerados.
    Cambiar al en el la limpieza de datos, no quitar stopwords, etc.
    Cambrias los hiperparametros de los objetos CountVectorizer, MinHashLSH, etc.
    Usar otra linea para los vecinos mas cercanos, etc.
